!curl https://course.fast.ai/setup/colab | bash

### Fully connected forward and backward pass

In [2]:
#export
from exp.nb_01 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [2]:
x_train,y_train,x_val,y_val = get_data()

In [3]:
train_mean, train_std = x_train.mean(),x_train.std()
train_mean, train_std

(tensor(0.1304), tensor(0.3073))

In [4]:
x_train.shape

torch.Size([50000, 784])

In [5]:
x_train = normalize(x_train,train_mean,train_std)
x_val = normalize(x_val,train_mean,train_std)

In [6]:
train_mean, train_std = x_train.mean(),x_train.std()
train_mean, train_std

(tensor(0.0001), tensor(1.))

In [7]:
#export 
def test_near_zero(a,tol=1e-3): assert a.abs()<tol, f"Near zero: {a}"

In [8]:
test_near_zero(x_train.mean())
test_near_zero(1-x_train.std())

In [9]:
n,m=x_train.shape
c=y_train.max()+1
n,m,c

(50000, 784, tensor(10))

### Foundations 

#### Basic architecture (1 hidden layer)

In [10]:
#number of hidden
nh = 50

In [11]:
# simplified kaiming init/he init
w1 = torch.randn(m,nh)/math.sqrt(m)
b1 = torch.randn(nh)
w2 = torch.randn(nh,1)/math.sqrt(nh)
b2 = torch.randn(1)

In [12]:
test_near_zero(w1.mean())
test_near_zero(w1.std()-1/math.sqrt(m))

In [13]:
# export
def lin(x,w,b): return x@w+b

In [14]:
t=lin(x_val,w1,b1)
t.mean(),t.std()

(tensor(-0.0315), tensor(1.3914))

In [15]:
def relu(x): return x.clamp_min(0.)

In [16]:
t=relu(lin(x_val,w1,b1))
t.mean(), t.std()

(tensor(0.5383), tensor(0.8067))

In [17]:
w1 = torch.randn(m,nh)*math.sqrt(2/m)
w1.mean(),w1.std()

(tensor(6.3423e-05), tensor(0.0508))

In [18]:
t=relu(lin(x_val,w1,b1))
t.mean(),t.std()

(tensor(0.6292), tensor(0.9955))

In [19]:
#export
from torch.nn import init

In [20]:
w1 = torch.zeros(m,nh)
init.kaiming_normal_(w1,mode='fan_out')
t = relu(lin(x_val,w1,b1))
t.mean(),t.std()

(tensor(0.6636), tensor(0.9793))

In [21]:
init.kaiming_normal_??

In [22]:
w1.shape

torch.Size([784, 50])

In [23]:
import torch.nn

In [24]:
torch.nn.Linear(m,nh).weight.shape

torch.Size([50, 784])

In [25]:
torch.nn.Linear.forward??

In [26]:
torch.nn.functional.linear??

In [27]:
torch.nn.Conv2d??

In [28]:
torch.nn.modules.conv._ConvNd.reset_parameters??

In [29]:
# jeremy test on moving the average back
def relu(x): return x.clamp_min(0.)-0.5

In [30]:
w1 = torch.randn(m,nh)*math.sqrt(2./m)
t1 = relu(lin(x_val,w1,b1))
t1.mean(),t1.std()

(tensor(0.1537), tensor(0.9733))

In [31]:
def model(xb):
    l1 = lin(xb,w1,b1)
    l2 = relu(l1)
    l3 = lin(l2,w2,b2)
    return l3

In [32]:
%timeit -n 10 _=model(x_val)

8.12 ms ± 486 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [33]:
assert model(x_val).shape==torch.Size([x_val.shape[0],1])

#### loss function: MSE (things simplified by using MSE but its useless)

In [34]:
model(x_val).shape

torch.Size([10000, 1])

In [35]:
def mse(output,targ): return (output.squeeze(1)-targ).pow(2).mean()

In [36]:
y_train,y_val = y_train.float(),y_val.float()

In [37]:
preds=model(x_train)

In [38]:
preds.shape

torch.Size([50000, 1])

In [39]:
mse(preds,y_train)

tensor(20.8757)

### Backward pass

In [40]:
def mse_grad(inp, targ):
    # grad of loss with respect to previou layer
    inp.g = 2.*(inp.squeeze()-targ).unsqueeze(-1)/inp.shape[0]

In [41]:
def relu_grad(inp, out):
    # grad of relu with respect to input activations
    inp.g = (inp>0).float()*out.g

In [42]:
def lin_grad(inp,out,w,b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1)*out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [43]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    # we dont need the loss in backward
    loss = mse(out, targ)
    
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1,l2)
    lin_grad(inp,l1,w1,b1)

In [44]:
forward_and_backward(x_train,y_train)

In [45]:
# save for testing purposes later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig = x_train.g.clone()

In [46]:
# (cheating) use pytorch autograd to check out the results
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [47]:
def forward(inp, targ):
    # forward pass:
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    # we dont need the loss in backward
    return mse(out, targ)

In [48]:
loss=forward(xt2,y_train)

In [49]:
loss.backward()

In [50]:
test_near(w22.grad,w2g)
test_near(b22.grad,b2g)
test_near(w12.grad,w1g)
test_near(b12.grad,b1g)
test_near(xt2.grad,ig)

### Refactoring

#### layers and classes

In [51]:
class Relu():
    def __call__(self,inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)-0.5
        return self.out
    def backward(self):
        self.inp.g = (self.inp>0).float()*self.out.g

In [52]:
class Lin():
    def __init__(self,w,b): self.w,self.b=w,b
        
    def __call__(self,inp):
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        # creating a gigant outer product just to sum it - inefficient
        self.w.g = (self.inp.unsqueeze(-1)*self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [53]:
class Mse():
    def __call__(self,inp,targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze()-targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2.*(self.inp.squeeze()-self.targ).unsqueeze(-1)/self.targ.shape[0]

In [54]:
class Model():
    def __init__(self,w1,b1,w2,b2):
        self.layers = [Lin(w1,b1),Relu(),Lin(w2,b2)]
        self.loss = Mse()
    def __call__(self,x,targ):
        for l in self.layers: x = l(x)
        return self.loss(x,targ)
    def  backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [55]:
w1.g,b1.g,w2.g,b2.g=[None]*4
model = Model(w1,b1,w2,b2)

In [56]:
%time loss=model(x_train,y_train)

CPU times: user 284 ms, sys: 45.6 ms, total: 330 ms
Wall time: 60.8 ms


In [57]:
%time model.backward()

CPU times: user 6.23 s, sys: 2.3 s, total: 8.53 s
Wall time: 8.57 s


In [58]:
test_near(w2g,w2.g)
test_near(b2g,b2.g)
test_near(w1g,w1.g)
test_near(b1g,b1.g)
test_near(ig,x_train.g)

#### refactor 2 (with ensum and abstract Module class)

In [59]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)

In [60]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)-0.5
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [61]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = torch.einsum("bi,bj->ij", inp, out.g)
        self.b.g = out.g.sum(0)

In [62]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [63]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [64]:
w1.g,b1.g,w2.g,b2.g=[None]*4
model = Model()

In [65]:
%time loss=model(x_train,y_train)

CPU times: user 294 ms, sys: 54.3 ms, total: 348 ms
Wall time: 51 ms


In [66]:
%time model.backward()

CPU times: user 459 ms, sys: 60.4 ms, total: 520 ms
Wall time: 144 ms


In [67]:
test_near(w2g,w2.g)
test_near(b2g,b2.g)
test_near(w1g,w1.g)
test_near(b1g,b1.g)
test_near(ig,x_train.g)

#### without einsum

In [68]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.t()@out.g
        self.b.g = out.g.sum(0)

In [69]:
w1.g,b1.g,w2.g,b2.g=[None]*4
model = Model()

In [70]:
%time loss=model(x_train,y_train)

CPU times: user 296 ms, sys: 51.3 ms, total: 348 ms
Wall time: 50.6 ms


In [71]:
%time model.backward()

CPU times: user 470 ms, sys: 58.9 ms, total: 529 ms
Wall time: 144 ms


### nn.Linear and nn.Module

In [72]:
#export
from torch import nn

In [79]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        self.loss = Mse
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x.squeeze(), targ)

In [80]:
model = Model(m,nh,1)

In [81]:
%time loss=model(x_train,y_train)

TypeError: Mse() takes no arguments

In [76]:
%time loss.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

## Export

In [7]:
!./notebook2script.py 02_Fully-connected.ipynb

Converted 02_Fully-connected.ipynb to exp/nb_02.py
